First, we need a 'chessboard' to hold numbers.

In [1]:
from collections import deque,namedtuple
from random import choice
from tqdm import tqdm
import numpy as np

In [2]:
PUZZLE_DIM = 3 
Action = namedtuple('Action', ['pos1', 'pos2'])

TARGET_STATE = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))


In [3]:

def available_actions(state: np.ndarray) -> list[Action]:
    """Get all possible exchange moves for the current board state"""
    x, y = [int(_[0]) for _ in np.where(state == 0)]  # Get the position of the space (0)
    actions = list()
    # Determine if it can move up, down, left and right
    if x > 0:
        actions.append(Action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(Action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(Action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(Action((x, y), (x, y + 1)))
    return actions

def do_action(state: np.ndarray, action: Action) -> np.ndarray:
    """Swap two elements"""
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

In [4]:
def generate_puzzle(num_moves=100_000) -> np.ndarray:
    """Randomly generate a disrupted board state"""
    state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
    for _ in tqdm(range(num_moves), desc="Randomizing"):
        actions = available_actions(state)
        state = do_action(state, choice(actions))  # Execute a random action
    return state

In [5]:
initial_state = generate_puzzle(num_moves=160)

print("Initial state:")
print(initial_state)

Randomizing: 100%|██████████| 160/160 [00:00<00:00, 151044.03it/s]

Initial state:
[[6 1 8]
 [4 0 7]
 [2 5 3]]


Now, get a random 'chessboard' and then I used bfs to solve the problem

In [6]:
def bfs(initial_state: np.ndarray) -> list[Action]:
    
    target_state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
    visited = set()
    queue = deque([(initial_state, [], 0)])  # stores the current state and the path to that state
    action_count = 0

    while queue:
        state, path, cost = queue.popleft()

        if np.array_equal(state, target_state):
            print("Solution found!")
            print(f"Total number of actions evaluated (cost): {action_count}")
            return path

        state_key = tuple(state.flatten())  
        if state_key in visited:
            continue
        visited.add(state_key)

        actions = available_actions(state)
        for act in actions:
            new_state = do_action(state, act)
            if tuple(new_state.flatten()) not in visited:
                queue.append((new_state, path + [act], cost + 1))
                action_count += 1

    return []

In [7]:
solution = bfs(initial_state)
if solution:
    print(f"Solution found with {len(solution)} moves!")
    print("Solution path:")
    for move in solution:
        print(f"Move: {move}")
        
else:
    print("No solution found.")

Solution found!
Total number of actions evaluated (cost): 199660
Solution found with 24 moves!
Solution path:
Move: Action(pos1=(1, 1), pos2=(2, 1))
Move: Action(pos1=(2, 1), pos2=(2, 2))
Move: Action(pos1=(2, 2), pos2=(1, 2))
Move: Action(pos1=(1, 2), pos2=(1, 1))
Move: Action(pos1=(1, 1), pos2=(2, 1))
Move: Action(pos1=(2, 1), pos2=(2, 0))
Move: Action(pos1=(2, 0), pos2=(1, 0))
Move: Action(pos1=(1, 0), pos2=(0, 0))
Move: Action(pos1=(0, 0), pos2=(0, 1))
Move: Action(pos1=(0, 1), pos2=(1, 1))
Move: Action(pos1=(1, 1), pos2=(2, 1))
Move: Action(pos1=(2, 1), pos2=(2, 2))
Move: Action(pos1=(2, 2), pos2=(1, 2))
Move: Action(pos1=(1, 2), pos2=(0, 2))
Move: Action(pos1=(0, 2), pos2=(0, 1))
Move: Action(pos1=(0, 1), pos2=(1, 1))
Move: Action(pos1=(1, 1), pos2=(1, 0))
Move: Action(pos1=(1, 0), pos2=(2, 0))
Move: Action(pos1=(2, 0), pos2=(2, 1))
Move: Action(pos1=(2, 1), pos2=(2, 2))
Move: Action(pos1=(2, 2), pos2=(1, 2))
Move: Action(pos1=(1, 2), pos2=(1, 1))
Move: Action(pos1=(1, 1), pos2=(

This BFS is a “dumb” approach, and it is not as efficient when n > 3.